In [1]:
import sys
sys.path.append('..')
import os
import sklearn.metrics.cluster
import sklearn.cluster
import seaborn as sns
import scanpy as sc
import sklearn as sk
from sklearn.neighbors import NearestNeighbors
import matplotlib as mpl
from matplotlib import pyplot as plt
import skdim
import seaborn as sns
import time
import numpy as np
import pandas as pd
import pickle
import rpy2.robjects as ro
import rpy2.robjects.numpy2ri
import rpy2.robjects.packages as rpackages
import utils
import anndata2ri
import gc
import benchmark_common_functions
from itertools import product

anndata2ri.activate()
ro.numpy2ri.activate()
readRDS = ro.r['readRDS']
%load_ext rpy2.ipython

## Already processed datasets

In [4]:
rootdir = '/mnt/c/Users/jobac/Desktop/all/singlecell-id-paper/data/Cytotrace_datasets/anndata/'
resdir = '../results/Cytotrace_datasets/hubness_results/'

fnames = np.array(os.listdir(rootdir))
sort_fnames = np.argsort(list(map(lambda fname : int(fname.split('_')[1].split('.')[0]),fnames)))
sorted_fnames = fnames[sort_fnames]

get_res_path = lambda fname: f'{resdir}{fname}_norm{do_norm}_scale{norm_scale}_pca{do_pca}_ncomps{n_comps}_{metric}_weighted{weighted}_knn{n_neighbors}_seed{seed}.pkl'

In [5]:
#fixed params
do_log = False #already done with do_norm
do_pca = True
n_neighbors = 10
seed = 0

#vary params
metric=('cosine','euclidean')
n_comps=(25,50,100,500)
weighted=(True,)
do_norm = (None,)
norm_scale = (False,)
params_list = list(product(metric,n_comps,weighted,do_norm,norm_scale))
# params_list = [i for i in params_list if not(i[-2]=='duo' and i[-1]==True)]

In [12]:
for metric,n_comps,weighted,do_norm,norm_scale in params_list:
    print(metric,n_comps,weighted,do_norm,norm_scale)
    for data_idx in np.arange(len(sorted_fnames)):
        if get_res_path(sorted_fnames[data_idx][:-5]).split('/')[-1] in os.listdir(resdir):
            continue
        print(data_idx,end=' ')
        adata = sc.read(rootdir+sorted_fnames[data_idx]) 

        retained_cells_idx = np.where(np.isin(adata.uns['allcellnames'],adata.uns['cellnames']))[0] if data_idx not in [0,29,32] else range(len(adata.X))

        results_dict = benchmark_common_functions.clustering_analysis(
            adata,
            retained_cells_idx=retained_cells_idx,
            true_labels = adata.uns['Phenotype'],
            do_norm = do_norm,
            norm_scale = norm_scale,
            do_log = do_log,
            do_pca = do_pca,
            n_neighbors= n_neighbors,
            n_clusters=len(np.unique(adata.uns['Phenotype'])),
            metric=metric,
            weighted=weighted,  #weighted adjmat for louvain/leiden clustering ?
            seed=seed,
            n_comps=n_comps)

        with open(get_res_path(sorted_fnames[data_idx][:-5]),'wb') as f:
            pickle.dump(results_dict,f)
        %xdel adata
        gc.collect()

cosine 25 True None False
cosine 50 True None False
cosine 100 True None False
cosine 500 True None False
euclidean 25 True None False
euclidean 50 True None False
euclidean 100 True None False
euclidean 500 True None False


# Raw datasets

In [2]:
rootdir = '/mnt/c/Users/jobac/Desktop/all/singlecell-id-paper/data/Cytotrace_raw_datasets/anndata/'
resdir = '../results/Cytotrace_raw_datasets/hubness_results/'

fnames = np.array(os.listdir(rootdir))
sort_fnames = np.argsort(list(map(lambda fname : int(fname.split('_')[2].split('.')[0]),fnames)))
sorted_fnames = fnames[sort_fnames]

proc_rootdir = '/mnt/c/Users/jobac/Desktop/all/singlecell-id-paper/data/Cytotrace_datasets/anndata/'
proc_fnames = np.array(os.listdir(proc_rootdir))
proc_sort_fnames = np.argsort(list(map(lambda fname : int(fname.split('_')[1].split('.')[0]),proc_fnames)))
proc_sorted_fnames = proc_fnames[sort_fnames]

#get_res_path = lambda fname: f'{resdir}{fname}_norm{do_norm}_scale{norm_scale}_pca{do_pca}_ncomps{n_comps}_{metric}_weighted{weighted}_knn{n_neighbors}_seed{seed}.pkl'
get_res_path = lambda fname: f'{resdir}{fname}_norm{do_norm}_scale{norm_scale}_pca{do_pca}_ncomps{n_comps}_{metric}_weighted{weighted}_knn_sqrt_seed{seed}.pkl'
get_res_path_sc = lambda fname: f'{resdir}{fname}_norm{do_norm}_scale{norm_scale}_pca{do_pca}_ncomps{n_comps}_{metric}_weighted{weighted}_knn_sqrt_seed{seed}_scanpyhubness.pkl'

In [3]:
#fixed params
do_log = True #already done with do_norm
do_pca = True
#n_neighbors = 10
seed = 0

#vary params
metric=('cosine','euclidean')
n_comps=(25,50,100,500)
weighted=(True,)
do_norm = ('duo','seurat')#,'zheng17')
norm_scale = (True,False)
params_list = list(product(metric,n_comps,weighted,do_norm,norm_scale))
# params_list = [i for i in params_list if not(i[-2]=='duo' and i[-1]==True)]

In [11]:
fails = []
for metric,n_comps,weighted,do_norm,norm_scale in params_list:
    print(metric,n_comps,weighted,do_norm,norm_scale)
    for data_idx in np.arange(len(sorted_fnames)):
        if get_res_path(sorted_fnames[data_idx][:-5]).split('/')[-1] in os.listdir(resdir):
            continue
        print(data_idx,end=' ')
        adata = sc.read(rootdir+sorted_fnames[data_idx]) 
        proc_adata = sc.read(proc_rootdir+proc_sorted_fnames[data_idx]) 
        retained_cells_idx = np.where(np.isin(proc_adata.uns['allcellnames'],proc_adata.uns['cellnames']))[0]
        print('\t',round(np.sqrt(len(adata))))
        #try:
        results_dict = benchmark_common_functions.clustering_analysis(
            adata,
            retained_cells_idx=retained_cells_idx,
            true_labels = proc_adata.uns['Phenotype'],
            do_norm = do_norm,
            norm_scale = norm_scale,
            do_log = do_log,
            do_pca = do_pca,
            n_neighbors= int(round(np.sqrt(len(adata)))),
            n_clusters=len(np.unique(proc_adata.uns['Phenotype'])),
            metric=metric,
            weighted=weighted,  #weighted adjmat for louvain/leiden clustering ?
            seed=seed,
            n_comps=n_comps)

        with open(get_res_path(sorted_fnames[data_idx][:-5]),'wb') as f:
            pickle.dump(results_dict,f)
        #except:
        #    fails.append([metric,n_comps,weighted,do_norm,norm_scale,sorted_fnames[data_idx]])
        %xdel adata
        gc.collect()

cosine 25 True duo True
0 	 40
		duo norm retained 5000 genes
			Preprocessing done: 0.02 mn
			Inputs generated: 0.01 mn
			Scanpy louvain/leiden clus: 0.13 mn
			Louvain/leiden clus: 0.0 mn
			Scanpy louvain/leiden clus, searching ground truth: 0.07 mn
			Base louvain/leiden clus, searching ground truth: 0.01 mn
			Scoring: 0.0 mn
			Hubness methods full pipeline: 0.12 mn
1 	 21
		duo norm retained 5000 genes
			Preprocessing done: 0.01 mn
			Inputs generated: 0.01 mn
			Scanpy louvain/leiden clus: 0.01 mn
			Louvain/leiden clus: 0.0 mn
			Scanpy louvain/leiden clus, searching ground truth: 0.01 mn
			Base louvain/leiden clus, searching ground truth: 0.0 mn
			Scoring: 0.0 mn
			Hubness methods full pipeline: 0.03 mn
2 	 21
		duo norm retained 5000 genes
			Preprocessing done: 0.01 mn
			Inputs generated: 0.0 mn
			Scanpy louvain/leiden clus: 0.01 mn
			Louvain/leiden clus: 0.0 mn
Cannot find the number of clusters
Clustering solution from last iteration is used:12 at resolution 2.99

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/jo/anaconda3/envs/w/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-11-d37d6153a2b4>", line 26, in <module>
    n_comps=n_comps)
  File "/mnt/c/Users/jobac/Desktop/all/singlecell-id/notebooks/cluster_utils.py", line 408, in clustering_analysis
    res_key['clus']['scanpy_'+method]['leiden'],  res_key['clus_info']['scanpy_'+method]['leiden']  = getNclusters(adata, G, n_clusters=n_clusters, seed=seed,cluster_func='leiden',flavor='scanpy',weights=weights)
  File "/mnt/c/Users/jobac/Desktop/all/singlecell-id/notebooks/cluster_utils.py", line 144, in getNclusters
    sc.tl.leiden(adata,resolution=this_resolution,random_state=seed,use_weights=weighted)
  File "/home/jo/anaconda3/envs/w/lib/python3.7/site-packages/scanpy/tools/_leiden.py", line 144, in leiden
    part = leidenalg.find_partition(g, partition_type, **partition_kwargs

KeyboardInterrupt: 

In [4]:
fails = []
for metric,n_comps,weighted,do_norm,norm_scale in params_list:
    print(metric,n_comps,weighted,do_norm,norm_scale)
    for data_idx in np.arange(len(sorted_fnames)):
        #if get_res_path_sc(sorted_fnames[data_idx][:-5]).split('/')[-1] in os.listdir(resdir):
        #    continue
        print(data_idx,end=' ')
        adata = sc.read(rootdir+sorted_fnames[data_idx]) 
        proc_adata = sc.read(proc_rootdir+proc_sorted_fnames[data_idx]) 
        retained_cells_idx = np.where(np.isin(proc_adata.uns['allcellnames'],proc_adata.uns['cellnames']))[0]
        print('\t',round(np.sqrt(len(adata))))
        #try:
        results_dict = benchmark_common_functions.scanpy_hubness_analysis(
            adata,
            retained_cells_idx=retained_cells_idx,
            do_norm = do_norm,
            norm_scale = norm_scale,
            do_log = do_log,
            do_pca = do_pca,
            n_neighbors= int(round(np.sqrt(len(adata)))),
            metric=metric,
            weighted=weighted,  #weighted adjmat for louvain/leiden clustering ?
            seed=seed,
            n_comps=n_comps)

        with open(get_res_path_sc(sorted_fnames[data_idx][:-5]),'wb') as f:
            pickle.dump(results_dict,f)
        #except:
        #    fails.append([metric,n_comps,weighted,do_norm,norm_scale,sorted_fnames[data_idx]])
        %xdel adata
        gc.collect()

cosine 25 True duo True
0 	 40
		duo norm retained 5000 genes
			Scoring: 0.12 mn
1 	 21
		duo norm retained 5000 genes
			Scoring: 0.02 mn
2 	 21
		duo norm retained 5000 genes
			Scoring: 0.04 mn
3 	 15
		duo norm retained 5000 genes
			Scoring: 0.02 mn
4 	 77
		duo norm retained 5000 genes
			Scoring: 0.53 mn
5 	 77
		duo norm retained 5000 genes
			Scoring: 0.26 mn
6 	 12
		duo norm retained 5000 genes
			Scoring: 0.01 mn
7 	 27
		duo norm retained 5000 genes
			Scoring: 0.07 mn
8 	 19
		duo norm retained 5000 genes
			Scoring: 0.02 mn
9 	 15
		duo norm retained 5000 genes
			Scoring: 0.02 mn
10 	 16
		duo norm retained 5000 genes
			Scoring: 0.01 mn
11 	 38
		duo norm retained 5000 genes
			Scoring: 0.06 mn
12 	 20
		duo norm retained 5000 genes
			Scoring: 0.01 mn
13 	 21
		duo norm retained 5000 genes
			Scoring: 0.03 mn
14 	 32
		duo norm retained 5000 genes
			Scoring: 0.06 mn
15 	 13
		duo norm retained 5000 genes
			Scoring: 0.01 mn
16 	 22
		duo norm retained 5000 genes
			